# Most Dangerous Streets - Month to Month Graph

Make a chart over time to show how the most dangerous streets have changed over time. 

In [1]:
import pytz
import folium
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from matplotlib._color_data import BASE_COLORS, TABLEAU_COLORS, CSS4_COLORS, XKCD_COLORS

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
query = """
with route_centerline_count as (
    select
    sr.gid
    , sr.lrsroute
    , sc.fullname
    , row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
        as fullname_priority

    from street_routes sr
    inner join street_centerline sc using (lrsroute)

    group by 1,2,3
)

, crashes_routes as (
    select distinct
    case when sr.gid = 3901 then 3900 else sr.gid end as gid
    , c.incident_id
    , to_char(c.reported_date, 'YYYY-MM') as crash_year_month
    , c.sbi
    , c.fatality
    , c.sbi_or_fatality

    from street_routes sr
    inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

    where not at_freeway
    
    -- Exclude this month's crashes
    and to_char(c.reported_date, 'YYYY-MM') < to_char(current_timestamp, 'YYYY-MM')
    
    -- Start on July 1, 2020
    and to_char(c.reported_date, 'YYYY-MM') >= '2020-07'
)

, count_routes as (
    select
    cr.gid
    , sr1.lrsroute
    , st_length(sr1.geom_denver) / 5280 as length_miles
    , cr.crash_year_month
    , count(cr.incident_id) as num_crashes
    , sum(cr.sbi::int) as num_sbi
    , sum(cr.fatality::int) as num_fatality
    , sum(cr.sbi_or_fatality::int) as num_sbi_or_fatality
    
    from crashes_routes cr
    inner join street_routes sr1 using (gid)
    
    group by 1,2,3,4
)

select
count_routes.gid
, rcc.fullname
, count_routes.length_miles
, count_routes.crash_year_month
, count_routes.num_crashes
, count_routes.num_sbi
, count_routes.num_fatality
, count_routes.num_sbi_or_fatality

from count_routes
inner join route_centerline_count rcc on (rcc.gid = count_routes.gid and rcc.fullname_priority = 1)

order by count_routes.gid, count_routes.crash_year_month
"""

In [4]:
street_month_crashes = pd.read_sql(query, cda.conn)

In [5]:
median_month_crashes = street_month_crashes.groupby(
    ['gid', 'fullname']).num_crashes.median().reset_index(name='median_num_crashes')

In [11]:
highest_median_crashes = median_month_crashes.sort_values(by='median_num_crashes', ascending=False).head(30)
highest_median_crashes

,gid,fullname,median_num_crashes
518,1012,E COLFAX AVE,28.0
520,1016,N COLORADO BLVD,22.0
1964,4689,N FEDERAL BLVD,20.0
1521,3900,N SPEER BLVD,19.0
447,880,N BROADWAY,18.0
2998,7584,S FEDERAL BLVD,17.0
1902,4592,S COLORADO BLVD,16.5
2717,6796,W ALAMEDA AVE,15.0
2569,6476,S SANTA FE DR,14.0
1119,2474,N QUEBEC ST,14.0


In [12]:
list_of_street_gids_to_plot = list(highest_median_crashes['gid'])

In [14]:
for street_gid_to_plot in list_of_street_gids_to_plot:
    fig, ax = plt.subplots(figsize=(16,6))

    data_to_plot = street_month_crashes[street_month_crashes.gid == street_gid_to_plot]

    this_street_median = median_month_crashes.loc[
        median_month_crashes.gid == street_gid_to_plot, 'median_num_crashes'
    ].values[0]

    this_street_name = median_month_crashes.loc[
        median_month_crashes.gid == street_gid_to_plot, 'fullname'
    ].values[0]
    
    this_street_filename = this_street_name.replace(' ','_')

    ax.bar(
        x=data_to_plot['crash_year_month']
        , height=data_to_plot['num_crashes']
        , color=TABLEAU_COLORS['tab:red']
    )

    ax.plot(
        data_to_plot['crash_year_month']
        , len(data_to_plot) * [this_street_median]
        , color='black'
        , linestyle=':'
    )

    ax.tick_params(axis='x', rotation=70)

    n = 3  # Keep every Nth label
    [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

    plt.ylabel('Number of Crashes')
    plt.title(f'Crashes by month: {this_street_name} (gid: {street_gid_to_plot})')
    plt.savefig(
        f'images/streets/crashes_by_month_{this_street_filename}_{street_gid_to_plot}.png'
        , transparent=False, facecolor='white', bbox_inches='tight'
    )
    plt.close()

## All crashes in Denver by month

When did crash rate stabilize after the pandemic begun? Approx July 1, 2020

In [9]:
# denver_crashes_by_month = street_month_crashes.groupby('crash_year_month').num_crashes.sum()

# fig, ax = plt.subplots(figsize=(16,6))

# ax.bar(
#     x=denver_crashes_by_month.index
#     , height=denver_crashes_by_month
#     , color=TABLEAU_COLORS['tab:red']
# )

# ax.tick_params(axis='x', rotation=70)

# n = 6  # Keep every Nth label
# [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

# plt.ylabel('Number of Crashes')
# plt.title(f'Crashes by month: Denver')
# plt.savefig('images/denver_crashes_by_month.png', transparent=False, facecolor='white', bbox_inches='tight')